In [11]:
import pandas as pd
import numpy as np

In [12]:
# Hücre 2: Ana veri setinden gerekli sütunları seçerek yeni bir DataFrame oluşturur
maindata = pd.read_excel("data/Hackathon_MainData.xlsx")

# Gerekli sütunlarla yeni bir DataFrame oluştur
df = pd.DataFrame()

df["Enlem"] = maindata["lat"]
df["Boylam"] = maindata["lng"]
df["İlçe"] = maindata["MusteriBolge4"]
df["Tür"] = maindata["SatisKanali"]
df["Modern Değeri"] = maindata["DDegeri"]
df["Traditional Değeri"] = maindata["RDegeri"]
df["Hotel Değeri"] = maindata["HDegeri"]
df["Ortalama Harcama Tutarı"] = maindata["OrtalamaHarcamaTutari"]
df["Restoran Çeşidi"] = maindata["MusteriCesidi"]
df["Map Profile Score"] = maindata["MapProfileScore"]
df["Map Population Score"] = maindata["MapPopulationScore"]
df["Mapin Segment"] = maindata["Mapin Segment"]

In [13]:
# Hücre 3: Yanlış yazılmış bölge ve kanal değerlerini düzeltir
# İlçe sütunundaki ATASEHIR ve ÜMRANIYE değerlerini doğru Türkçe karakterlerle değiştirir
# Tür sütunundaki TRADİTİONAL hatasını düzeltir
# Restoran Çeşidi sütunundaki 'Balik' kelimesini 'Balık' olarak düzeltir

df["İlçe"] = df["İlçe"].replace({
    "ATASEHIR": "ATAŞEHİR",
    "ÜMRANIYE": "ÜMRANİYE"
})
df["Tür"] = df["Tür"].replace({
    "TRADİTİONAL": "TRADITIONAL",
})
df["Restoran Çeşidi"] = df['Restoran Çeşidi'].str.replace("Balik", "Balık", regex=False)

In [14]:
# Hücre 4: Modern, Traditional ve Hotel değerlerindeki başlıktaki harfleri kaldırır

df['Modern Değeri'] = df['Modern Değeri'].str.replace('D', '', regex=False)
df['Traditional Değeri'] = df['Traditional Değeri'].str.replace('R', '', regex=False)
df['Hotel Değeri'] = df['Hotel Değeri'].str.replace('H', '', regex=False)

In [15]:
# Hücre 5: Enlem, Boylam, Map Profile ve Map Population skorlarını sayı formatına çevirir
# Beş haneli sayıları üç haneli yapacak şekilde bölme işlemi uygular

def format_float(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        value = value.replace(",", ".")
        value = float(value)
    try:
        float_value = value
        if 10000 <= float_value < 100000:
            return float_value / 1000
        else:
            return float_value
    except ValueError:
        return np.nan

# Uygulama
[df_col for df_col in ["Map Profile Score", "Map Population Score", "Enlem", "Boylam"]]
for col in ["Map Profile Score", "Map Population Score", "Enlem", "Boylam"]:
    df[col] = df[col].apply(format_float)

In [16]:
# %%
# Hücre 6: Ortalama Harcama Tutarı sütunundaki değerleri sayısal hale getirir
# Aralık belirtan değerlerde ortalama, plus işaretlerinde alt sınır kullanılır

def ortalama_harcama_to_num(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = x.replace(' TL', '').replace('.', '')
    if '-' in x:
        alt, ust = x.split('-')
        alt = int(alt)
        ust = int(ust)
        return round((alt + ust) / 2)
    elif '+' in x:
        sayi = int(x.replace('+', ''))
        return sayi
    else:
        return int(x)
    
# Fonksiyonu uygulama
[df_col for df_col in ["Ortalama Harcama Tutarı"]]
df["Ortalama Harcama Tutarı"] = df["Ortalama Harcama Tutarı"].apply(ortalama_harcama_to_num)

In [17]:
# Hücre 7: Modern, Traditional ve Hotel değerlerindeki boş (NaN) değerleri 0 ile doldurur

df['Modern Değeri'] = df['Modern Değeri'].fillna(0)
df['Traditional Değeri'] = df['Traditional Değeri'].fillna(0)
df['Hotel Değeri'] = df['Hotel Değeri'].fillna(0)

In [18]:
# Hücre 8: Son haliyle DataFrame'i ekrana yazdırır

df

,Enlem,Boylam,İlçe,Tür,Modern Değeri,Traditional Değeri,Hotel Değeri,Ortalama Harcama Tutarı,Restoran Çeşidi,Map Profile Score,Map Population Score,Mapin Segment
0,40.995444,29.024803,KADIKÖY,HOTEL,5,0,2,750.0,Business,NaN,NaN,H2
1,40.980917,29.101372,ATAŞEHİR,HOTEL,5,0,1,2000.0,"Banket, Event, Etkinlik Oteli",NaN,NaN,H1
2,41.054086,29.051875,ÜSKÜDAR,HOTEL,5,0,1,750.0,"Diğer (Apart, Pansiyon)",NaN,NaN,H1
3,41.019103,28.957953,ÜSKÜDAR,MODERN,4,0,0,750.0,Tekne-Event Hall-Catering,67.1815,54.2895,D4-C
4,40.965051,29.072068,KADIKÖY,MODERN,3,0,0,750.0,Modern Pub & Bistro,77.7405,62.0554,D3-B
...,...,...,...,...,...,...,...,...,...,...,...,...
781,40.976207,29.047311,KADIKÖY,TRADITIONAL,0,0,0,2000.0,Geleneksel Meyhane,73.4570,66.8130,R3-B
782,40.986261,29.033062,KADIKÖY,MODERN,0,0,0,1500.0,"Dg.Mdrn(Tekne,S.Ev,S.house,Ct)",73.3350,64.1790,D3-B
783,40.940802,29.111243,KADIKÖY,TRADITIONAL,0,0,0,2000.0,Kebap,NaN,NaN,R2-B
784,41.002037,29.154804,ATAŞEHİR,TRADITIONAL,0,0,0,1500.0,Kebap,NaN,NaN,R3-B


In [19]:
df.to_csv("MainData_updated.csv", index=False)

In [20]:
# Sütunları listele
columns = df.columns.tolist()
for i in range(len(columns)):
    print(columns[i])

Enlem
Boylam
İlçe
Tür
Modern Değeri
Traditional Değeri
Hotel Değeri
Ortalama Harcama Tutarı
Restoran Çeşidi
Map Profile Score
Map Population Score
Mapin Segment
